In [220]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import math
import random
import time
from datetime import timedelta
import matplotlib.pyplot as plt

In [221]:
def flip(p):
    return 1 if random.random() < p else 0

In [261]:
def gen_pattern1(numbr_entries,freq_day = 3, final_death_days = 60, consistent_pattern_per = 0.7, decay = 'smooth_and_final_death', repeat_random = False):
    
    
    consistent_pattern_per = min(consistent_pattern_per,(numbr_entries - final_death_days - 7)/numbr_entries)
    numbr_consistent_units = math.floor(numbr_entries * consistent_pattern_per/freq_day)
    consistent_len = consistent_pattern_per*numbr_entries
    
    consistent_units_all = []
    if repeat_random == False:
        unit_pattern = [1]+[0]*(freq_day - 1)
        consistent_units_all = unit_pattern * numbr_consistent_units
    else:
        if freq_day <= consistent_len:
            consistent_units_all = [1]+[0]*(freq_day - 1)
            len_all_units = len(consistent_units_all)
            while(len_all_units<(consistent_len - (freq_day + 3))):
                add_pattern = [1]+[0]*random.randint((freq_day - 2),(freq_day +2))
                consistent_units_all = consistent_units_all + add_pattern
                len_all_units = len(consistent_units_all)
     
    
    if decay == 'sharp':
        remaining_units_all = [0] * (numbr_entries - len(consistent_units_all))
        total_pattern = consistent_units_all + remaining_units_all
    
    if decay == 'smooth_and_final_death':
        decay_period = numbr_entries - final_death_days - len(consistent_units_all)
        if(decay_period <0):
            print(f"Decay period {decay_period}")
            print(f'consistent_pattern_len {consistent_len}')
            print(f'consistent_units_all {len(consistent_units_all)}')
            
        decay_x = np.linspace(1/freq_day,0,decay_period)
        decay_pattern = list(map(flip,decay_x))

        remaining_units_all = [0] * final_death_days
        total_pattern = consistent_units_all + decay_pattern + remaining_units_all
    return total_pattern

In [262]:
def str_time_prop(start, end, format, prop):

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)


In [263]:
numbr_dataset = 5

In [264]:
def gen_pattern2(numbr_entries, p):
    return list(map(flip,p))

In [276]:
def gen_table(numbr_dataset):

    start_dates = [random_date(start = '2018-01-01', end = '2020-01-01', prop = random.random()) for i in range(numbr_dataset)]
    start_dates = [datetime.strptime(date,'%Y-%m-%d') for date in start_dates]
    end_dates = ['2021-01-01' for i in range(numbr_dataset)]
    end_dates = [datetime.strptime(date,'%Y-%m-%d') for date in end_dates]
    freq_sequence = [1,1,2,2,3,4,5,6,7,7,14,30,45,60,90,120,180,240]
    freq_list = [random.choice(freq_sequence) for i in range(numbr_dataset)]
    death_days_list = [random.randint(10,45) for i in range(numbr_dataset)]
    consistent_pattern_list = [random.randint(50,90)/100 for i in range(numbr_dataset)]
    decay_sequence = ['sharp','smooth_and_final_death']
    decay_list = [random.choice(decay_sequence) for i in range(numbr_dataset)]
    repeat_sequence = [False, True]
    repeat_list = [random.choice(repeat_sequence) for i in range(numbr_dataset)]

    df_list = []
    for i in range(numbr_dataset):
        freq_day = freq_list[i]
        final_death_days = death_days_list[i]
        consistent_pattern_per = consistent_pattern_list[i]
        decay = decay_list[i]
        repeat_random = repeat_list[i]
        start_date = start_dates[i]
        end_date = end_dates[i]
        time_series = pd.date_range(start = start_date,end=end_date)
        numbr_entries = len(time_series)
        
        pattern_choice = random.choice(['random','pattern', 'pattern', 'pattern', 'pattern'])
        if pattern_choice == 'pattern':
            total_pattern = gen_pattern1(numbr_entries, freq_day, final_death_days, consistent_pattern_per, decay, repeat_random)
        else:
            p = [1/random.randint(5,120)] * numbr_entries
            total_pattern = gen_pattern2(numbr_entries, p)
        
        blob_key_list = [i]*len(time_series)
        print(f'Nmbr Entries {numbr_entries} Pattern Length {len(total_pattern)} Blob Key List {len(blob_key_list)}' )
        
        size_multiplier = [0.5, 0.7,0.8, 0.85, 0.9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.05, 1.1, 1.1, 1.15, 1.3]
        size_mult_list = [random.choice(size_multiplier) for i in range(numbr_entries)]
        size_start = random.randint(10000, 100000)
        size_list = [size_start]
        
        modified_time_list = [start_date]
        
        for i in range(1, numbr_entries):
            if((size_mult_list[i-1] != 1) & (total_pattern[i] == 1)):
                modified_time_list.append(time_series[i])
                size = math.ceil(size_list[i-1] * size_mult_list[i-1])
            
            else:
                modified_time_list.append( modified_time_list[i-1])
                size = size_list[i-1]
            
            size_list.append(size)
        
        scanned_time_list = [date + timedelta(hours = 23) for date in time_series]
        
        ts_df = pd.DataFrame({'Access_Time':time_series, 'Accessed':total_pattern, 'Blob_Key':blob_key_list, 'Scanned_Time':scanned_time_list, 'Modified_Time':modified_time_list, 'Size':size_list})
        df_list.append(ts_df)

    table_df = pd.concat(df_list)
    table_df = table_df[table_df['Accessed'] == 1]
    table_df = table_df.drop(columns = 'Accessed')
    #size of dataset
    #size_list = np.random.randint(low = 10000, high = 100000, size = numbr_dataset)
    #Type of dataset
    file_type_list = ['CSV']*numbr_dataset
    #Container
    container_list = ['container1']*numbr_dataset
    #descriptor_df = pd.DataFrame({"Size":size_list, "File_Type":file_type_list, 'Blob_Key':range(numbr_dataset), 'Container':container_list})
    
    #File Path
    path_list = []
    
    for i in range(numbr_dataset):
        file_depth = random.randint(0,3)
        file_path = ''
        if file_depth >= 1:
            folder_list = ['Folder1', 'Folder2', 'Folder3']
            folder = random.choice(folder_list)
            file_path = file_path + str(folder) + '/'
        if file_depth >= 2:
            sub_folder_list = ['Sub_Folder1', 'Sub_Folder2', 'Sub_Folder3']
            sub_folder = random.choice(sub_folder_list)
            file_path = file_path + str(sub_folder) + '/'
        file_path = file_path + f'File_Num_{i}.csv'
        path_list.append(file_path)
        
    tier_list = ['Hot']*numbr_dataset
    container_list = ['Container1']*numbr_dataset
    
    descriptor_df = pd.DataFrame({"File_Type":file_type_list, 'Blob_Key':range(numbr_dataset), 'Blob_Path':path_list, 'Current_Tier':tier_list, 'Container':container_list})
    table_df = table_df.merge(descriptor_df, how = 'left', on = 'Blob_Key')
    
    Creation_Time_df = table_df.groupby('Blob_Key')['Access_Time'].min().to_frame().reset_index().rename(columns={'Access_Time':'Creation_Time'})
    table_df = table_df.merge(Creation_Time_df, how = 'left', on = 'Blob_Key')
    
    return table_df

In [277]:
numbr_dataset = 10000

In [278]:
table_df = gen_table(numbr_dataset)

Nmbr Entries 970 Pattern Length 970 Blob Key List 970
Nmbr Entries 832 Pattern Length 832 Blob Key List 832
Nmbr Entries 892 Pattern Length 892 Blob Key List 892
Nmbr Entries 530 Pattern Length 530 Blob Key List 530
Nmbr Entries 539 Pattern Length 539 Blob Key List 539
Nmbr Entries 707 Pattern Length 707 Blob Key List 707
Nmbr Entries 562 Pattern Length 562 Blob Key List 562
Nmbr Entries 413 Pattern Length 413 Blob Key List 413
Nmbr Entries 1021 Pattern Length 1021 Blob Key List 1021
Nmbr Entries 1004 Pattern Length 1004 Blob Key List 1004
Nmbr Entries 839 Pattern Length 839 Blob Key List 839
Nmbr Entries 909 Pattern Length 909 Blob Key List 909
Nmbr Entries 458 Pattern Length 458 Blob Key List 458
Nmbr Entries 518 Pattern Length 518 Blob Key List 518
Nmbr Entries 975 Pattern Length 975 Blob Key List 975
Nmbr Entries 1088 Pattern Length 1088 Blob Key List 1088
Nmbr Entries 855 Pattern Length 855 Blob Key List 855
Nmbr Entries 656 Pattern Length 656 Blob Key List 656
Nmbr Entries 988 Pa

Nmbr Entries 370 Pattern Length 370 Blob Key List 370
Nmbr Entries 876 Pattern Length 876 Blob Key List 876
Nmbr Entries 1091 Pattern Length 1091 Blob Key List 1091
Nmbr Entries 899 Pattern Length 899 Blob Key List 899
Nmbr Entries 431 Pattern Length 431 Blob Key List 431
Nmbr Entries 1091 Pattern Length 1091 Blob Key List 1091
Nmbr Entries 522 Pattern Length 522 Blob Key List 522
Nmbr Entries 681 Pattern Length 681 Blob Key List 681
Nmbr Entries 868 Pattern Length 868 Blob Key List 868
Nmbr Entries 512 Pattern Length 512 Blob Key List 512
Nmbr Entries 712 Pattern Length 712 Blob Key List 712
Nmbr Entries 437 Pattern Length 437 Blob Key List 437
Nmbr Entries 536 Pattern Length 536 Blob Key List 536
Nmbr Entries 852 Pattern Length 852 Blob Key List 852
Nmbr Entries 733 Pattern Length 733 Blob Key List 733
Nmbr Entries 986 Pattern Length 986 Blob Key List 986
Nmbr Entries 1046 Pattern Length 1046 Blob Key List 1046
Nmbr Entries 607 Pattern Length 607 Blob Key List 607
Nmbr Entries 717 Pa

Nmbr Entries 780 Pattern Length 780 Blob Key List 780
Nmbr Entries 819 Pattern Length 819 Blob Key List 819
Nmbr Entries 497 Pattern Length 497 Blob Key List 497
Nmbr Entries 1028 Pattern Length 1028 Blob Key List 1028
Nmbr Entries 1096 Pattern Length 1096 Blob Key List 1096
Nmbr Entries 860 Pattern Length 860 Blob Key List 860
Nmbr Entries 440 Pattern Length 440 Blob Key List 440
Nmbr Entries 837 Pattern Length 837 Blob Key List 837
Nmbr Entries 418 Pattern Length 418 Blob Key List 418
Nmbr Entries 760 Pattern Length 760 Blob Key List 760
Nmbr Entries 465 Pattern Length 465 Blob Key List 465
Nmbr Entries 968 Pattern Length 968 Blob Key List 968
Nmbr Entries 1092 Pattern Length 1092 Blob Key List 1092
Nmbr Entries 393 Pattern Length 393 Blob Key List 393
Nmbr Entries 801 Pattern Length 801 Blob Key List 801
Nmbr Entries 423 Pattern Length 423 Blob Key List 423
Nmbr Entries 657 Pattern Length 657 Blob Key List 657
Nmbr Entries 838 Pattern Length 838 Blob Key List 838
Nmbr Entries 889 Pa

Nmbr Entries 882 Pattern Length 882 Blob Key List 882
Nmbr Entries 487 Pattern Length 487 Blob Key List 487
Nmbr Entries 382 Pattern Length 382 Blob Key List 382
Nmbr Entries 467 Pattern Length 467 Blob Key List 467
Nmbr Entries 425 Pattern Length 425 Blob Key List 425
Nmbr Entries 1012 Pattern Length 1012 Blob Key List 1012
Nmbr Entries 1095 Pattern Length 1095 Blob Key List 1095
Nmbr Entries 429 Pattern Length 429 Blob Key List 429
Nmbr Entries 451 Pattern Length 451 Blob Key List 451
Nmbr Entries 768 Pattern Length 768 Blob Key List 768
Nmbr Entries 726 Pattern Length 726 Blob Key List 726
Nmbr Entries 1043 Pattern Length 1043 Blob Key List 1043
Nmbr Entries 738 Pattern Length 738 Blob Key List 738
Nmbr Entries 442 Pattern Length 442 Blob Key List 442
Nmbr Entries 479 Pattern Length 479 Blob Key List 479
Nmbr Entries 729 Pattern Length 729 Blob Key List 729
Nmbr Entries 588 Pattern Length 588 Blob Key List 588
Nmbr Entries 478 Pattern Length 478 Blob Key List 478
Nmbr Entries 774 Pa

Nmbr Entries 533 Pattern Length 533 Blob Key List 533
Nmbr Entries 742 Pattern Length 742 Blob Key List 742
Nmbr Entries 695 Pattern Length 695 Blob Key List 695
Nmbr Entries 409 Pattern Length 409 Blob Key List 409
Nmbr Entries 839 Pattern Length 839 Blob Key List 839
Nmbr Entries 1086 Pattern Length 1086 Blob Key List 1086
Nmbr Entries 893 Pattern Length 893 Blob Key List 893
Nmbr Entries 617 Pattern Length 617 Blob Key List 617
Nmbr Entries 637 Pattern Length 637 Blob Key List 637
Nmbr Entries 1066 Pattern Length 1066 Blob Key List 1066
Nmbr Entries 841 Pattern Length 841 Blob Key List 841
Nmbr Entries 394 Pattern Length 394 Blob Key List 394
Nmbr Entries 549 Pattern Length 549 Blob Key List 549
Nmbr Entries 1068 Pattern Length 1068 Blob Key List 1068
Nmbr Entries 779 Pattern Length 779 Blob Key List 779
Nmbr Entries 785 Pattern Length 785 Blob Key List 785
Nmbr Entries 636 Pattern Length 636 Blob Key List 636
Nmbr Entries 812 Pattern Length 812 Blob Key List 812
Nmbr Entries 464 Pa

...

Nmbr Entries 465 Pattern Length 465 Blob Key List 465
Nmbr Entries 1081 Pattern Length 1081 Blob Key List 1081
Nmbr Entries 920 Pattern Length 920 Blob Key List 920
Nmbr Entries 747 Pattern Length 747 Blob Key List 747
Nmbr Entries 715 Pattern Length 715 Blob Key List 715
Nmbr Entries 646 Pattern Length 646 Blob Key List 646
Nmbr Entries 1030 Pattern Length 1030 Blob Key List 1030
Nmbr Entries 582 Pattern Length 582 Blob Key List 582
Nmbr Entries 731 Pattern Length 731 Blob Key List 731
Nmbr Entries 1087 Pattern Length 1087 Blob Key List 1087
Nmbr Entries 377 Pattern Length 377 Blob Key List 377
Nmbr Entries 857 Pattern Length 857 Blob Key List 857
Nmbr Entries 459 Pattern Length 459 Blob Key List 459
Nmbr Entries 1025 Pattern Length 1025 Blob Key List 1025
Nmbr Entries 615 Pattern Length 615 Blob Key List 615
Nmbr Entries 809 Pattern Length 809 Blob Key List 809
Nmbr Entries 698 Pattern Length 698 Blob Key List 698
Nmbr Entries 978 Pattern Length 978 Blob Key List 978
Nmbr Entries 712

Nmbr Entries 576 Pattern Length 576 Blob Key List 576
Nmbr Entries 1016 Pattern Length 1016 Blob Key List 1016
Nmbr Entries 757 Pattern Length 757 Blob Key List 757
Nmbr Entries 806 Pattern Length 806 Blob Key List 806
Nmbr Entries 875 Pattern Length 875 Blob Key List 875
Nmbr Entries 437 Pattern Length 437 Blob Key List 437
Nmbr Entries 768 Pattern Length 768 Blob Key List 768
Nmbr Entries 777 Pattern Length 777 Blob Key List 777
Nmbr Entries 1062 Pattern Length 1062 Blob Key List 1062
Nmbr Entries 854 Pattern Length 854 Blob Key List 854
Nmbr Entries 829 Pattern Length 829 Blob Key List 829
Nmbr Entries 1006 Pattern Length 1006 Blob Key List 1006
Nmbr Entries 827 Pattern Length 827 Blob Key List 827
Nmbr Entries 536 Pattern Length 536 Blob Key List 536
Nmbr Entries 865 Pattern Length 865 Blob Key List 865
Nmbr Entries 838 Pattern Length 838 Blob Key List 838
Nmbr Entries 602 Pattern Length 602 Blob Key List 602
Nmbr Entries 479 Pattern Length 479 Blob Key List 479
Nmbr Entries 911 Pa

Nmbr Entries 371 Pattern Length 371 Blob Key List 371
Nmbr Entries 1070 Pattern Length 1070 Blob Key List 1070
Nmbr Entries 759 Pattern Length 759 Blob Key List 759
Nmbr Entries 916 Pattern Length 916 Blob Key List 916
Nmbr Entries 473 Pattern Length 473 Blob Key List 473
Nmbr Entries 516 Pattern Length 516 Blob Key List 516
Nmbr Entries 649 Pattern Length 649 Blob Key List 649
Nmbr Entries 1091 Pattern Length 1091 Blob Key List 1091
Nmbr Entries 612 Pattern Length 612 Blob Key List 612
Nmbr Entries 501 Pattern Length 501 Blob Key List 501
Nmbr Entries 1025 Pattern Length 1025 Blob Key List 1025
Nmbr Entries 720 Pattern Length 720 Blob Key List 720
Nmbr Entries 661 Pattern Length 661 Blob Key List 661
Nmbr Entries 923 Pattern Length 923 Blob Key List 923
Nmbr Entries 1013 Pattern Length 1013 Blob Key List 1013
Nmbr Entries 903 Pattern Length 903 Blob Key List 903
Nmbr Entries 1014 Pattern Length 1014 Blob Key List 1014
Nmbr Entries 695 Pattern Length 695 Blob Key List 695
Nmbr Entries 

Nmbr Entries 787 Pattern Length 787 Blob Key List 787
Nmbr Entries 620 Pattern Length 620 Blob Key List 620
Nmbr Entries 891 Pattern Length 891 Blob Key List 891
Nmbr Entries 822 Pattern Length 822 Blob Key List 822
Nmbr Entries 916 Pattern Length 916 Blob Key List 916
Nmbr Entries 594 Pattern Length 594 Blob Key List 594
Nmbr Entries 425 Pattern Length 425 Blob Key List 425
Nmbr Entries 458 Pattern Length 458 Blob Key List 458
Nmbr Entries 996 Pattern Length 996 Blob Key List 996
Nmbr Entries 1037 Pattern Length 1037 Blob Key List 1037
Nmbr Entries 490 Pattern Length 490 Blob Key List 490
Nmbr Entries 753 Pattern Length 753 Blob Key List 753
Nmbr Entries 875 Pattern Length 875 Blob Key List 875
Nmbr Entries 746 Pattern Length 746 Blob Key List 746
Nmbr Entries 417 Pattern Length 417 Blob Key List 417
Nmbr Entries 625 Pattern Length 625 Blob Key List 625
Nmbr Entries 680 Pattern Length 680 Blob Key List 680
Nmbr Entries 428 Pattern Length 428 Blob Key List 428
Nmbr Entries 620 Pattern 

Nmbr Entries 526 Pattern Length 526 Blob Key List 526
Nmbr Entries 1041 Pattern Length 1041 Blob Key List 1041
Nmbr Entries 403 Pattern Length 403 Blob Key List 403
Nmbr Entries 456 Pattern Length 456 Blob Key List 456
Nmbr Entries 795 Pattern Length 795 Blob Key List 795
Nmbr Entries 1087 Pattern Length 1087 Blob Key List 1087
Nmbr Entries 608 Pattern Length 608 Blob Key List 608
Nmbr Entries 694 Pattern Length 694 Blob Key List 694
Nmbr Entries 929 Pattern Length 929 Blob Key List 929
Nmbr Entries 802 Pattern Length 802 Blob Key List 802
Nmbr Entries 870 Pattern Length 870 Blob Key List 870
Nmbr Entries 504 Pattern Length 504 Blob Key List 504
Nmbr Entries 386 Pattern Length 386 Blob Key List 386
Nmbr Entries 837 Pattern Length 837 Blob Key List 837
Nmbr Entries 1020 Pattern Length 1020 Blob Key List 1020
Nmbr Entries 648 Pattern Length 648 Blob Key List 648


In [279]:
table_df.head()

,Access_Time,Blob_Key,Scanned_Time,Modified_Time,Size,File_Type,Blob_Path,Current_Tier,Container,Creation_Time
0,2018-05-08,0,2018-05-08 23:00:00,2018-05-08,82591,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08
1,2018-05-11,0,2018-05-11 23:00:00,2018-05-08,82591,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08
2,2018-05-13,0,2018-05-13 23:00:00,2018-05-13,94980,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08
3,2018-05-19,0,2018-05-19 23:00:00,2018-05-13,94980,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08
4,2018-05-25,0,2018-05-25 23:00:00,2018-05-25,85482,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08


In [280]:
recent_record = table_df.groupby(['Blob_Key'])['Access_Time'].max().to_frame().reset_index()

In [281]:
recent_record['Is_Recent_Record'] = 1

In [282]:
table_df = table_df.merge(recent_record, how = 'left', on = ['Blob_Key', 'Access_Time'])

In [283]:
table_df['Is_Recent_Record'] = table_df['Is_Recent_Record'].fillna(0)

In [284]:
table_df.head()

,Access_Time,Blob_Key,Scanned_Time,Modified_Time,Size,File_Type,Blob_Path,Current_Tier,Container,Creation_Time,Is_Recent_Record
0,2018-05-08,0,2018-05-08 23:00:00,2018-05-08,82591,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08,0.0
1,2018-05-11,0,2018-05-11 23:00:00,2018-05-08,82591,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08,0.0
2,2018-05-13,0,2018-05-13 23:00:00,2018-05-13,94980,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08,0.0
3,2018-05-19,0,2018-05-19 23:00:00,2018-05-13,94980,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08,0.0
4,2018-05-25,0,2018-05-25 23:00:00,2018-05-25,85482,CSV,Folder3/File_Num_0.csv,Hot,Container1,2018-05-08,0.0


In [285]:
table_df.to_csv(f'example_data3_{numbr_dataset}.csv')

In [286]:
table_df.to_parquet(f'example_data3_{numbr_dataset}.parquet')